In [1]:
%reload_ext cython

In [ ]:
! pip install stop_words

In [20]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [21]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [22]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer



#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):
    
    
    #Check to make sure NeedToSplit argument is 0 or 1
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        if isinstance(corpus[i], str) != True:
            print("Corpus input is not a string")
            return;
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
            #Remove stop words#
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]
            
            #remove stop words
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [23]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [24]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [25]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

In [26]:
k = 3
np.random.seed(7)
alphaOld = 10*np.random.rand(k)
V = corpusMatrix[0][0].shape[1]
betaOld = np.random.rand(k, V)
betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]

In [27]:
%%time 
for i in range(0,100):
    phi, gamma = Estep(k, 0, alphaOld, betaOld, corpusMatrix, 0.1)

Wall time: 112 ms


**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [28]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [29]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)

Check that gsl and scipy results are similar
----

In [30]:
! python setup.py build
! python setup.py install

python: can't open file 'setup.py': [Errno 2] No such file or directory
python: can't open file 'setup.py': [Errno 2] No such file or directory


In [31]:
%%cython -lgsl

#from cython_gsl cimport *
import cython_gsl
import scipy

print(gsl_sf_psi_n(1, 10))
print(scipy.special.polygamma(1, 10))
print(gsl_sf_psi(10))
print(scipy.special.psi(10))


Error compiling Cython file:
------------------------------------------------------------
...

#from cython_gsl cimport *
import cython_gsl
import scipy

print(gsl_sf_psi_n(1, 10))
                 ^
------------------------------------------------------------

C:\Users\Megan Robertson\.ipython\cython\_cython_magic_17ef91aa9981bf023b6149da5ea5f586.pyx:6:18: undeclared name not builtin: gsl_sf_psi_n

Error compiling Cython file:
------------------------------------------------------------
...
import cython_gsl
import scipy

print(gsl_sf_psi_n(1, 10))
print(scipy.special.polygamma(1, 10))
print(gsl_sf_psi(10))
               ^
------------------------------------------------------------

C:\Users\Megan Robertson\.ipython\cython\_cython_magic_17ef91aa9981bf023b6149da5ea5f586.pyx:8:16: undeclared name not builtin: gsl_sf_psi


Cython code for updateing $\alpha$ and $\beta$
----

In [ ]:
%%cython -a -lgsl
from cython_gsl cimport *

In [ ]:
%%cython -a -lgsl

import cython
import numpy as np
cimport numpy as np
import scipy
from cython_gsl cimport *

#Update alpha using linear Newton-Rhapson Method#

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:] cy_alphaUpdate(int k, int M, double[:] alphaOld, double[:, :] gamma, double tol):
    cdef double[:] h = np.zeros(k)
    cdef double[:] g = np.zeros(k)
    cdef double[:] alphaNew = np.zeros(k)

    cdef int converge
    cdef int i, d
    cdef double docSum
    cdef double[:] step = np.zeros(k)
    cdef double c, s1, s2

    cdef double alpha_sum = 0
    for i in range(k):
        alpha_sum += alphaOld[i]

    converge = 0
    while converge == 0:
        z = -gsl_sf_psi_n(1, alpha_sum)

        s1 = 0
        s2 = 1.0/z
        for i in range(0, k):
            docSum = 0
            for d in range(M):
                docSum += gsl_sf_psi(gamma[d,i]) - gsl_sf_psi(sum(gamma[d]))
            g[i] = M*(gsl_sf_psi(alpha_sum) - gsl_sf_psi(alphaOld[i])) + docSum
            h[i] = M*gsl_sf_psi_n(1, alphaOld[i])
            
            s1 += g[i]/h[i]
            s2 += 1.0/h[i]
        c = s1/s2

        for i in range(k):
            step[i] = (g[i] - c)/h[i]
        # step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            for i in range(k):
                alphaNew[i] = alphaOld[i] + step[i]
            alphaOld = alphaNew

    return alphaNew

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:, :] cy_betaUpdate(int k, int M, long[:] phi_1, double[:, :, :] phi_2, double[:, :] gamma, 
                              long[:] c_1, double[:, :, :] c_2, double tol):

    cdef int i, j, d, n
    cdef int Nd
    cdef double wordSum

    #Calculate beta#
    cdef int V = c_2[0].shape[1]
    cdef double[:, :] beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(M):
                Nd = c_1[d] # c[d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi_2[d, n,i]*c_2[d, n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]

    return beta

Check Cython functions against original functions
----

### Prepare funciton arguments

In [ ]:
k = 3
M = 4
phi = [np.array([[ 0.38340797,  0.30327435,  0.31331768],
       [ 0.5703127 ,  0.17742331,  0.25226399],
       [ 0.3176045 ,  0.43558053,  0.24681497]]), np.array([[ 0.55072546,  0.18583053,  0.26344401],
       [ 0.30048528,  0.44698122,  0.2525335 ],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608]]), np.array([[ 0.0623253 ,  0.24856995,  0.68910475],
       [ 0.63063084,  0.08148213,  0.28788703],
       [ 0.12858014,  0.39114008,  0.48027977]]), np.array([[ 0.66366397,  0.08257639,  0.25375964],
       [ 0.77627072,  0.10684157,  0.11688771],
       [ 0.75940435,  0.23665379,  0.00394186],
       [ 0.02553949,  0.62915851,  0.345302  ],
       [ 0.14168349,  0.41504784,  0.44326867]])]
gamma = np.array([np.array([ 9.71859516,  6.63758257,  7.30234283]), np.array([ 10.58244789,   7.50992466,   8.56614802]), np.array([ 9.26880627,  6.44249655,  7.94721775]), np.array([ 10.813832  ,   7.19158249,   7.65310607])])
alphaOld = np.array([ 8.44726999 , 5.72130438,  6.48994619])
c = np.array([np.array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])])
tol = 0.1

In [ ]:
phi

In [ ]:
r = len(phi)
phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
n = max(phi_1)
p = phi[0].shape[1]
phi_2 = np.zeros((r, n, p))
for i, j in enumerate(phi_1):
    phi_2[i, :j, :] = phi[i]

In [ ]:
c

In [ ]:
r = len(c)
c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
n = max(c_1)
p = c[0].shape[1]
c_2 = np.zeros((r, n, p))
for i, j in enumerate(c_1):
    c_2[i, :j, :] = c[i]

### Check

In [ ]:
a, b = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)

In [ ]:
a

In [ ]:
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
alphaNew

In [ ]:
b

In [ ]:
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
betaNew

Is there any speed-up?
----

In [ ]:
%timeit a, b = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)

In [ ]:
%%timeit
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))

**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [15]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol):

    # create rectangular matrices for c
    c = corpusMatrix[0]
    r_c = len(c)
    c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
    n_c = max(c_1)
    p_c = c[0].shape[1]
    c_2 = np.zeros((r_c, n_c, p_c))
    for i, j in enumerate(c_1):
        c_2[i, :j, :] = c[i]
    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        # create rectangular matrices for phi
        r = len(phi)
        phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
        n = max(phi_1)
        p = phi[0].shape[1]
        phi_2 = np.zeros((r, n, p))
        for i, j in enumerate(phi_1):
            phi_2[i, :j, :] = phi[i]

                
        gamma = np.array(gamma)

        alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
        betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [16]:
%%time

LDA(3, corpusMatrix, 0.1, 1)

TypeError: LDA() takes 3 positional arguments but 4 were given

In [ ]:
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    for i in range(0, k):
        document = betaDF.loc[i,:]
        document.sort(1, ascending = 0)
        mostCommon = pd.DataFrame(document[0:p])
        topicWords.append(mostCommon)
    return(topicWords)

In [ ]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 
docsTest = collections.OrderedDict()
docsTest["s1"] = doc_a
docsTest["s2"] = doc_b
docsTest["s3"] = doc_c
docsTest["s4"] = doc_d 
docsTest["s5"] = doc_e 
for k, v in docsTest.items():
    print(k,v)
    

In [ ]:
cTest = make_word_matrix(docsTest, 1)

**State of the Union Example**

In order to determine the initial performance time of our algorithm, we implemented LDA on a small corpus of eight documents. These documents are each of the State of the Union Adresses delivered by President Clinton during his time in office. These were found in the nltk Python package.

In [ ]:
import nltk
#nltk.download("state_union")
from nltk.corpus import state_union

In [ ]:
#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:58]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [ ]:
fileNames = state_union.fileids()
SOTU = fileNames

PresSOTU = {}
for i in range(0, len(SOTU)):
    PresSOTU[SOTU[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(SOTU[i]))

In [ ]:
np.random.seed(17)
PresCorp = make_word_matrix(PresSOTU, 0)
ResultsAll = LDA(5, PresCorp, 0.1)

In [ ]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, ClintCorp, 0.1)

In [ ]:
mostCommon(ResultsAll[0][3], PresCorp[1], 2)

In [ ]:
from nltk.corpus import gutenberg
gutenberg.fileids()

In [ ]:
bookNames = gutenberg.fileids()
indicies = [0,7,15]
books = [bookNames[i] for i in indicies]
books

In [ ]:


booksInput = {}
for i in range(0, len(books)):
    booksInput[books[i].rsplit('.', 1)[0]] = list(nltk.corpus.gutenberg.words(books[i]))

In [ ]:
np.random.seed(77)
bookCorp = make_word_matrix(booksInput, 0)
print("LDA")
ResultsBooks = LDA(5, bookCorp, 0.1)

## Sources:
http://www.gutenberg.org/cache/epub/4/pg4.txt
http://www.gutenberg.org/files/16780/16780-h/16780-h.html
http://www.americanrhetoric.com/speeches/mlkihaveadream.htm
http://www.americanrhetoric.com/speeches/jfkinaugural.htm

In [ ]:
Gettysburg = """Four score and seven years ago, our fathers brought forth
upon this continent a new nation:  conceived in liberty, and
dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war. . .testing whether
that nation, or any nation so conceived and so dedicated. . .
can long endure.  We are met on a great battlefield of that war.

We have come to dedicate a portion of that field as a final resting place
for those who here gave their lives that this nation might live.
It is altogether fitting and proper that we should do this.

But, in a larger sense, we cannot dedicate. . .we cannot consecrate. . .
we cannot hallow this ground.  The brave men, living and dead,
who struggled here have consecrated it, far above our poor power
to add or detract.  The world will little note, nor long remember,
what we say here, but it can never forget what they did here.

It is for us the living, rather, to be dedicated here to the unfinished
work which they who fought here have thus far so nobly advanced.
It is rather for us to be here dedicated to the great task remaining
before us. . .that from these honored dead we take increased devotion
to that cause for which they gave the last full measure of devotion. . .
that we here highly resolve that these dead shall not have died in vain. . .
that this nation, under God, shall have a new birth of freedom. . .
and that government of the people. . .by the people. . .for the people. . .
shall not perish from this earth.
"""

In [ ]:
Declaration = """The unanimous Declaration of the thirteen united States of America

When in the Course of human events, it becomes necessary for one people to dissolve the political bands 
which have connected them with another, and to assume, among the Powers of the earth, the separate and equal 
station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of 
mankind requires that they should 
declare the causes which impel them to the separation.

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their 
Creator with certain unalienable Rights, that among these are Life, Liberty, and the pursuit of Happiness.
—That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent 
of the governed,—That whenever any Form of Government becomes destructive of these ends, it is the Right 
of the People to alter or to abolish it, and to institute new Government, laying its foundation on such 
principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety 
and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for 
light and transient causes; and accordingly all experience hath shown, 
that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the 
forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same 
Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off 
such Government, and to provide new Guards for their future security.—Such has been the patient sufferance of these 
Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. 
The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in 
direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to 
a candid world.

He has refused his Assent to Laws, the most wholesome and necessary for the public good.
He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their 
operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.
He has refused to pass other Laws for the accommodation of large districts of people, unless those people would 
relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.
He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of 
their Public Records, for the sole purpose of fatiguing them into compliance with his measures.
He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights 
of the people.
He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative 
Powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining 
in the mean time exposed to all the dangers of invasion from without, and convulsions within.
He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws of 
Naturalization of Foreigners; refusing to pass others to encourage their migration hither, and raising the 
conditions of new Appropriations of Lands.
He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary Powers.
He has made judges dependent on his Will alone, for the tenure of their offices, and the amount and payment 
of their salaries.
He has erected a multitude of New Offices, and sent hither swarms of Officers to harass our People, and eat 
out their substance.
He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.
He has affected to render the Military independent of and superior to the Civil Power.
He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged 
by our laws; giving his Assent to their Acts of pretended legislation:
For quartering large bodies of armed troops among us:
For protecting them, by a mock Trial, from Punishment for any Murders which they should commit on the 
Inhabitants of these States:
For cutting off our Trade with all parts of the world:
For imposing taxes on us without our Consent:
For depriving us, in many cases, of the benefits of Trial by Jury:
For transporting us beyond Seas to be tried for pretended offences:
For abolishing the free System of English Laws in a neighbouring Province, establishing therein an 
Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit 
instrument for introducing the same absolute rule into these Colonies:
For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms 
of our Governments:
For suspending our own Legislatures, and declaring themselves invested with Power to legislate for us 
in all cases whatsoever.
He has abdicated Government here, by declaring us out of his Protection and waging War against us.
He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.
He is at this time transporting large armies of foreign mercenaries to compleat the works of death, 
desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled 
in the most barbarous ages, and totally unworthy of the Head of a civilized nation.
He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their 
Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.
He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of 
our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction 
of all ages, sexes and conditions.
In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated 
Petitions have been answered only by repeated injury. A Prince, whose character is thus marked by every act 
which may define a Tyrant, is unfit to be the ruler of a free People.

Nor have We been wanting in attention to our Brittish brethren. We have warned them from time to time of 
attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the 
circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, 
and we have conjured them by the ties of our common kindred to disavow these usurpations, which would 
inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice 
and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, 
and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.

We, therefore, the Representatives of the United States of America, in General Congress, Assembled, 
appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and 
by the Authority of the good People of these Colonies, solemnly publish and declare, That these United 
Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all 
Allegiance to the British Crown, and that all political connection between them and the State of Great 
Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full 
Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and 
Things which Independent States may of right do. And for the support of this Declaration, with a firm 
reliance on the Protection of Divine Providence, we mutually pledge to each other our Lives, our Fortunes 
and our sacred Honor.

"""

In [ ]:
MLK = """I am happy to join with you today in what will go down in history as the greatest 
demonstration for freedom in the history of our nation.

Five score years ago, a great American, in whose symbolic shadow we stand today, signed the 
Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions 
of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous 
daybreak to end the long night of their captivity.

But one hundred years later, the Negro still is not free. One hundred years later, the life of the 
Negro is still sadly crippled by the manacles of segregation and the chains of discrimination. One hundred 
years later, the Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity. 
One hundred years later, the Negro is still languished in the corners of American society and finds himself 
an exile in his own land. And so we've come here today to dramatize a shameful condition.

In a sense we've come to our nation's capital to cash a check. When the architects of our republic wrote 
the magnificent words of the Constitution and the Declaration of Independence, they were signing a promissory 
note to which every American was to fall heir. This note was a promise that all men, yes, black men as well 
as white men, would be guaranteed the "unalienable Rights" of "Life, Liberty and the pursuit of Happiness." 
It is obvious today that America has defaulted on this promissory note, insofar as her citizens of color are 
concerned. Instead of honoring this sacred obligation, America has given the Negro people a bad check, a check 
which has come back marked "insufficient funds."

But we refuse to believe that the bank of justice is bankrupt. We refuse to believe that there are insufficient 
funds in the great vaults of opportunity of this nation. And so, we've come to cash this check, a check that will 
give us upon demand the riches of freedom and the security of justice.

We have also come to this hallowed spot to remind America of the fierce urgency of Now. This is no time to
engage in the luxury of cooling off or to take the tranquilizing drug of gradualism. Now is the time to make 
real the promises of democracy. Now is the time to rise from the dark and desolate valley of segregation to 
the sunlit path of racial justice. Now is the time to lift our nation from the quicksands of racial injustice 
to the solid rock of brotherhood. Now is the time to make justice a reality for all of God's children.

It would be fatal for the nation to overlook the urgency of the moment. This sweltering summer of the Negro's 
legitimate discontent will not pass until there is an invigorating autumn of freedom and equality. Nineteen 
sixty-three is not an end, but a beginning. And those who hope that the Negro needed to blow off steam and 
will now be content will have a rude awakening if the nation returns to business as usual. And there will be 
neither rest nor tranquility in America until the Negro is granted his citizenship rights. The whirlwinds of 
revolt will continue to shake the foundations of our nation until the bright day of justice emerges.

But there is something that I must say to my people, who stand on the warm threshold which leads into the 
palace of justice: In the process of gaining our rightful place, we must not be guilty of wrongful deeds. 
Let us not seek to satisfy our thirst for freedom by drinking from the cup of bitterness and hatred. We must 
forever conduct our struggle on the high plane of dignity and discipline. We must not allow our creative 
protest to degenerate into physical violence. Again and again, we must rise to the majestic heights of 
meeting physical force with soul force.

The marvelous new militancy which has engulfed the Negro community must not lead us to a distrust of 
all white people, for many of our white brothers, as evidenced by their presence here today, have come to 
realize that their destiny is tied up with our destiny. And they have come to realize that their freedom 
is inextricably bound to our freedom.

We cannot walk alone.

And as we walk, we must make the pledge that we shall always march ahead.

We cannot turn back.

There are those who are asking the devotees of civil rights, "When will you be satisfied?" 
We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality.
We can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging 
in the motels of the highways and the hotels of the cities. *We cannot be satisfied as long as the 
negro's basic mobility is from a smaller ghetto to a larger one. We can never be satisfied as long as 
our children are stripped of their self-hood and robbed of their dignity by signs stating: "For Whites Only." 
We cannot be satisfied as long as a Negro in Mississippi cannot vote and a Negro in New York believes he has 
nothing for which to vote. No, no, we are not satisfied, and we will not be satisfied until "justice rolls 
down like waters, and righteousness like a mighty stream."¹

I am not unmindful that some of you have come here out of great trials and tribulations. 
Some of you have come fresh from narrow jail cells. And some of you have come from areas where your 
quest -- quest for freedom left you battered by the storms of persecution and staggered by the winds of 
police brutality. You have been the veterans of creative suffering. Continue to work with the faith that 
unearned suffering is redemptive. Go back to Mississippi, go back to Alabama, go back to South Carolina, 
go back to Georgia, go back to Louisiana, go back to the slums and ghettos of our northern cities, knowing 
that somehow this situation can and will be changed.

Let us not wallow in the valley of despair, I say to you today, my friends.

And so even though we face the difficulties of today and tomorrow, I still have a dream. It is a dream 
deeply rooted in the American dream.

I have a dream that one day this nation will rise up and live out the true meaning of its creed: "We hold 
these truths to be self-evident, that all men are created equal."

I have a dream that one day on the red hills of Georgia, the sons of former slaves and the sons of former 
slave owners will be able to sit down together at the table of brotherhood.

I have a dream that one day even the state of Mississippi, a state sweltering with the heat of injustice, 
sweltering with the heat of oppression, will be transformed into an oasis of freedom and justice.

I have a dream that my four little children will one day live in a nation where they will not be judged by 
the color of their skin but by the content of their character.

I have a dream today!

I have a dream that one day, down in Alabama, with its vicious racists, with its governor having his lips 
dripping with the words of "interposition" and "nullification" -- one day right there in Alabama little 
black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers.

I have a dream today!

I have a dream that one day every valley shall be exalted, and every hill and mountain shall be made 
low, the rough places will be made plain, and the crooked places will be made straight; "and the glory 
of the Lord shall be revealed and all flesh shall see it together."

This is our hope, and this is the faith that I go back to the South with.

With this faith, we will be able to hew out of the mountain of despair a stone of hope. With this faith, 
we will be able to transform the jangling discords of our nation into a beautiful symphony of brotherhood. 
With this faith, we will be able to work together, to pray together, to struggle together, to go to jail 
together, to stand up for freedom together, knowing that we will be free one day.

And this will be the day -- this will be the day when all of God's children will be able to sing 
with new meaning:

My country 'tis of thee, sweet land of liberty, of thee I sing.

Land where my fathers died, land of the Pilgrim's pride,

From every mountainside, let freedom ring!

 

And if America is to be a great nation, this must become true.

And so let freedom ring from the prodigious hilltops of New Hampshire.

Let freedom ring from the mighty mountains of New York.

Let freedom ring from the heightening Alleghenies of Pennsylvania.

Let freedom ring from the snow-capped Rockies of Colorado.

Let freedom ring from the curvaceous slopes of California.

 

But not only that:

Let freedom ring from Stone Mountain of Georgia.

Let freedom ring from Lookout Mountain of Tennessee.

Let freedom ring from every hill and molehill of Mississippi.

From every mountainside, let freedom ring.

 

And when this happens, and when we allow freedom ring, when we let it ring from every 
village and every hamlet, from every state and every city, we will be able to speed up that 
day when all of God's children, black men and white men, Jews and Gentiles, Protestants and 
Catholics, will be able to join hands and sing in the words of the old Negro spiritual:

                Free at last! Free at last!

                Thank God Almighty, we are free at last!"""

In [ ]:
JFK = """Vice President Johnson, Mr. Speaker, Mr. Chief Justice, President Eisenhower, Vice President Nixon, 
President Truman, reverend clergy, fellow citizens:

We observe today not a victory of party, but a celebration of freedom -- symbolizing an end, as well as a 
beginning -- signifying renewal, as well as change. For I have sworn before you and Almighty God the same 
solemn oath our forebears prescribed nearly a century and three-quarters ago.

The world is very different now. For man holds in his mortal hands the power to abolish all forms of human 
poverty and all forms of human life. And yet the same revolutionary beliefs for which our forebears fought 
are still at issue around the globe -- the belief that the rights of man come not from the generosity of 
the state, but from the hand of God.

We dare not forget today that we are the heirs of that first revolution. Let the word go forth from this 
time and place, to friend and foe alike, that the torch has been passed to a new generation of Americans -- 
born in this century, tempered by war, disciplined by a hard and bitter peace, proud of our ancient heritage, 
and unwilling to witness or permit the slow undoing of those human rights to which this nation has always 
been committed, and to which we are committed today at home and around the world.

Let every nation know, whether it wishes us well or ill, that we shall pay any price, bear any burden, 
meet any hardship, support any friend, oppose any foe, to assure the survival and the success of liberty.

This much we pledge -- and more.

To those old allies whose cultural and spiritual origins we share, we pledge the loyalty of faithful 
friends. United there is little we cannot do in a host of cooperative ventures. Divided there is little 
we can do -- for we dare not meet a powerful challenge at odds and split asunder.

To those new states whom we welcome to the ranks of the free, we pledge our word that one form of colonial 
control shall not have passed away merely to be replaced by a far more iron tyranny. We shall not always 
expect to find them supporting our view. But we shall always hope to find them strongly supporting their 
own freedom -- and to remember that, in the past, those who foolishly sought power by riding the back of 
the tiger ended up inside.

To those people in the huts and villages of half the globe struggling to break the bonds of mass misery, 
we pledge our best efforts to help them help themselves, for whatever period is required -- not because 
the Communists may be doing it, not because we seek their votes, but because it is right. If a free 
society cannot help the many who are poor, it cannot save the few who are rich.

To our sister republics south of our border, we offer a special pledge: to convert our good words into 
good deeds, in a new alliance for progress, to assist free men and free governments in casting off the 
chains of poverty. But this peaceful revolution of hope cannot become the prey of hostile powers. Let 
all our neighbors know that we shall join with them to oppose aggression or subversion anywhere in the 
Americas. And let every other power know that this hemisphere intends to remain the master of its own house.

To that world assembly of sovereign states, the United Nations, our last best hope in an age where the 
instruments of war have far outpaced the instruments of peace, we renew our pledge of support -- to 
prevent it from becoming merely a forum for invective, to strengthen its shield of the new and the 
weak, and to enlarge the area in which its writ may run.

Finally, to those nations who would make themselves our adversary, we offer not a pledge but a request: 
that both sides begin anew the quest for peace, before the dark powers of destruction unleashed by 
science engulf all humanity in planned or accidental self-destruction.

We dare not tempt them with weakness. For only when our arms are sufficient beyond doubt can we be 
certain beyond doubt that they will never be employed.

But neither can two great and powerful groups of nations take comfort from our present course -- 
both sides overburdened by the cost of modern weapons, both rightly alarmed by the steady spread of 
the deadly atom, yet both racing to alter that uncertain balance of terror that stays the hand of mankind's final war.
So let us begin anew -- remembering on both sides that civility is not a sign of weakness, and sincerity is 
always subject to proof. Let us never negotiate out of fear, but let us never fear to negotiate.

Let both sides explore what problems unite us instead of belaboring those problems which divide us.

Let both sides, for the first time, formulate serious and precise proposals for the inspection and control of 
arms, and bring the absolute power to destroy other nations under the absolute control of all nations.

Let both sides seek to invoke the wonders of science instead of its terrors. Together let us explore the stars, 
conquer the deserts, eradicate disease, tap the ocean depths, and encourage the arts and commerce.

Let both sides unite to heed, in all corners of the earth, the command of Isaiah -- to "undo the heavy burdens, 
and [to] let the oppressed go free."

And, if a beachhead of cooperation may push back the jungle of suspicion, let both sides join in 
creating a new endeavor -- not a new balance of power, but a new world of law -- where the strong are 
just, and the weak secure, and the peace preserved.

All this will not be finished in the first one hundred days. Nor will it be finished in the first one 
thousand days; nor in the life of this Administration; nor even perhaps in our lifetime on this planet. 
But let us begin.

In your hands, my fellow citizens, more than mine, will rest the final success or failure of our course. 
Since this country was founded, each generation of Americans has been summoned to give testimony to its 
national loyalty. The graves of young Americans who answered the call to service surround the globe.

Now the trumpet summons us again -- not as a call to bear arms, though arms we need -- not as a call to 
battle, though embattled we are -- but a call to bear the burden of a long twilight struggle, year in 
and year out, "rejoicing in hope; patient in tribulation,"² a struggle against the common enemies of man: 
tyranny, poverty, disease, and war itself.

Can we forge against these enemies a grand and global alliance, North and South, East and West, that 
can assure a more fruitful life for all mankind? Will you join in that historic effort?
In the long history of the world, only a few generations have been granted the role of defending freedom 
in its hour of maximum danger. I do not shrink from this responsibility -- I welcome it. I do not believe 
that any of us would exchange places with any other people or any other generation. The energy, 
the faith, the devotion which we bring to this endeavor will light our country and all who serve it. 
And the glow from that fire can truly light the world.

And so, my fellow Americans, ask not what your country can do for you; ask what you can do for 
your country.

My fellow citizens of the world, ask not what America will do for you, but what together we 
can do for the freedom of man.

Finally, whether you are citizens of America or citizens of the world, ask of us here the 
same high standards of strength and sacrifice which we ask of you. With a good conscience our 
only sure reward, with history the final judge of our deeds, let us go forth to lead the land we 
love, asking His blessing and His help, but knowing that here on earth God's work must truly be our own.
"""

In [ ]:
speechDocs = collections.OrderedDict()
speechDocs["s1"] = Gettysburg
speechDocs["s2"] = Declaration
speechDocs["s3"] = MLK
speechDocs["s4"] = JFK

In [ ]:
#np.random.seed(77)
speechCorp = make_word_matrix(speechDocs, 1)
#print("LDA")
#ResultsBooks = LDA(5, bookCorp, 0.1)

In [ ]:
stopWords = get_stop_words('english')
stopWords

In [ ]:
speechCorp

In [ ]:
np.random.seed(17)


ResultsSpeech = LDA(3, speechCorp, 0.1)

In [ ]:
mostCommon(ResultsSpeech[0][3], speechCorp[1], 5)

## Color Code Text by Topic
<span style="color:red">Topic 1</span> 

<span style="color:blue">Topic 2</span> 

<span style="color:green">Topic 3</span> 

If a specific word is in more than one topic, I assigned it to that topic that gave that word the highest probability.

When in the <span style="color:blue">Course</span>  of <span style="color:green">human</span> <span style="color:red">events</span> , it <span style="color:red">becomes</span> <span style="color:green">necessary</span> for <span style="color:green">one</span> <span style="color:green">people</span> to <span style="color:green">dissolve</span> the <span style="color:green">political</span> <span style="color:red">bands</span> 
which have <span style="color:blue">connected</span> them with <span style="color:green">another</span>, and to <span style="color:blue">assume</span>, <span style="color:blue">among</span> the <span style="color:green">Powers</span> of the <span style="color:green">earth</span> , the <span style="color:green">separate</span> and <span style="color:green">equal</span> 
<span style="color:blue">station</span> to which the <span style="color:blue">Laws</span> of <span style="color:blue">Nature</span> and of <span style="color:blue">Nature's</span> <span style="color:green">God</span> <span style="color:green">entitle</span> them, a <span style="color:red">decent</span> <span style="color:red">respect</span> to the <span style="color:red">opinions</span> of 
<span style="color:red">mankind</span> <span style="color:green">requires</span> that they should 
<span style="color:green">declare</span> the <span style="color:red">causes</span> which <span style="color:red">impel</span> them to the <span style="color:green">separation</span>.

We <span style="color:blue">hold</span> these <span style="color:green">truths</span> to be <span style="color:blue">self-evident</span>, that all <span style="color:blue">men</span> are <span style="color:red">created</span> <span style="color:green">equal</span>, that they are <span style="color:blue">endowed</span> by their 
<span style="color:green">Creator</span> with <span style="color:blue">certain unalienable Rights</span>, that <span style="color:blue">among</span> these are <span style="color:blue">Life</span>, <span style="color:green">Liberty</span>, and the <span style="color:red">pursuit</span> of <span style="color:blue">Happiness</span>.

In [ ]:
decIntro = """When in the Course of human events, it becomes necessary for one people to dissolve the political bands 
which have connected them with another, and to assume, among the Powers of the earth, the separate and equal 
station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of 
mankind requires that they should 
declare the causes which impel them to the separation.

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their 
Creator with certain unalienable Rights, that among these are Life, Liberty, and the pursuit of Happiness."""
colorCode(ResultsSpeech[0][3], speechCorp[1], decIntro).T

In [ ]:
def colorCode(beta, wordList, inputWords):
    
    #Split text, convert to lowercase and remove punctuation and stop words
    stopWords = get_stop_words('english')
    table = dict.fromkeys(map(ord, string.punctuation))
    text = inputWords.translate(table)
    words = text.lower().split()
    words = [word for word in words if word not in stopWords]
    p_stemmer = PorterStemmer()
    words = [p_stemmer.stem(word) for word in words]
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    
    
    k = len(words)
    topicWords = np.zeros((1,k))
    topicWords = pd.DataFrame(topicWords,  columns=words)

    for i in range(0, k):
        document = np.array(betaDF.loc[:,words[i]])
        topicWords.iloc[:,i] = np.argmax(document)
    return(topicWords)

In [ ]:
mostCommon(ResultsSpeech[0][3], speechCorp[1], 25)

In [ ]:
stopWords

In [ ]:
#pip install --pre line-profiler
#pip install psutil
# pip install memory_profiler

In [ ]:
#creating profile files for the construction of the word matrix and LDA for SOTU example
%prun -q -D ClintonWordMatrix.prof make_word_matrix(ClintonSOTU,0)
%prun -q -D ClintonLDA.prof LDA(3, ClintCorp, 0.1, 0)

In [ ]:
#printing profile results
import pstats
wordMatrixProfile = pstats.Stats('ClintonWordMatrix.prof')
LDAProfile = pstats.Stats('ClintonLDA.prof')
wordMatrixProfile.print_stats()
LDAProfile.print_stats()
pass

**Movie Clustering** 

In addition to text analysis, LDA can be implemented for other problems that have the same structure as a text corpora. In this section, we use LDA in order to predict movies preferred by users of the site MovieLens.com. The goal is to predict another movie that the user likes based on the movies that they have already said that they prefer. The definition of preferred here is that the user rated the movie four or five, where five is the maxiumum possible rating. Users who gave a preferred rating to at least 50 movies were used. A training set of 2,535 users was used to fit the model. The model is then shown all but one of the preferred movies for the users in the test data, and this is used to determine the last movie that the user preferred. Thus, it is possible to evaluate the performance of the model.

In [17]:
#reading in the data
import pandas as pd
Ratings = pd.read_csv("RatingsDataClean.csv")

In [18]:
#making a dictionary for the training and testing data
Users = list(set(Ratings['userID']))
RatingsDict = {}
for i in range(0, len(Users)):
    movies = list(map(str, list(pd.DataFrame(Ratings[Ratings['userID'] == Users[0]])['movieID'])))
    RatingsDict[i] = ' '.join(movies)

In [19]:
#running LDA analysis to get parameters for movie training data
import random

random.seed(10)
movieMatrix = make_word_matrix(RatingsDict, 1)
movieOutput = LDA(6, movieMatrix, 0.1)

NameError: name 'cy_alphaUpdate' is not defined

In [ ]:
#use the above parameters to make predictions for the test set
#hold out one movie randomly, then determine the likelihood of the possible held out movies
#the highest likelihood should be assigned to the last movie, if the model works correctly

## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling

## Compare our output to Python Package

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

In [ ]:
#! pip install gensim

In [ ]:
p_stemmer = PorterStemmer()
for i in docs:
    stemmed_tokens = p_stemmer.stem(i)

In [ ]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "The quick brown fox"
doc_b = "Brown fox jumps over the jumps jumps jumps"
doc_c =  "The the the lazy dog elephant."
doc_d = "The the the the the dog peacock lion tiger elephant"


# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, iterations = 1)

In [ ]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

In [ ]:
beta =  LDATest(3, corpusMatrix, 0.1, 1)[0][3]

In [ ]:
mostCommon(beta, corpusMatrix[1], 4)

In [ ]:
corpusMatrix[1]

In [ ]:
beta

In [ ]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDATest(k, corpusMatrix, tol, needToSplit):

    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    iterations = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    

    phi = []
    gamma = []
    #looping through the number of documents
    for d in range(0,M): #M is the number of documents
        phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
        phi.append(phiT)
        gamma.append(gammaT)

    alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)


    converge =0
    alphaOld = alphaNew
    betaOld = betaNew
    iterations += iterations
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output